강의_6기_AI응용_6차시_01_GramMatrix과 Loss.ipynb

Style Transfer (스타일 전이)
- target: 학습이 도달해야 할 기준 (reference 정답)  
  - 생성되는 이미지(변하는 이미지) 따라가야 할 목표 상태(정답)

- content 목표 / style 목표
  - content 목표 : 이미지 속 형태나 구조 유지
    - target 콘텐츠 이미지의 feature map
  - style 목표: 이미지 가지고 있는 질감, 색감, 패턴 재현
     - target 스타일 이미지의 Gram Matrix

- 원본(style, content) 에서 추출된 features 정보
  - 생성 이미지가 학습을 통해 그 값과 가까워지는 것
  - loss가 0에 가까워 지는 지점 확보


In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision import transforms

from PIL import Image
from collections import OrderedDict

In [ ]:
# Gram Matrix
class GramMatrix(nn.Module):
  def forward(self, input):
    b,c,h,w = input.size()
    F = input.view(b, c, h*w) # flatten 

    # bmm : 배치 행렬곱
    # features: (b, c, h*w)
    # features.transpose(1,2): (b, h*w, c)
    # 출력 : (b, c, c) 
    G = torch.bmm(F, F.transpose(1,2))
    G.div_(h*w) # 정규화, 학습 목적으로 수식 단순하게 표현

    return G

In [ ]:
# GramMSELoss 정의
class GramMSELoss(nn.Module):
  def forward(self, input, target):
    out = nn.MSELoss()(GramMatrix()(input),target)
    return (out)

Content-style Loss

In [ ]:
class VGG(nn.Module):
    def __init__(self, pool='max'):
        super(VGG, self).__init__()
        #vgg modules
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        if pool == 'max':     # 특징 최대값 추출(뚜렷한 특징)
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pool == 'avg':  # 특징 평균 추출(부드러운 특징)
            self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x, out_keys):
        out = {}
        out['r11'] = F.relu(self.conv1_1(x))
        out['r12'] = F.relu(self.conv1_2(out['r11']))
        out['p1'] = self.pool1(out['r12'])
        out['r21'] = F.relu(self.conv2_1(out['p1']))
        out['r22'] = F.relu(self.conv2_2(out['r21']))
        out['p2'] = self.pool2(out['r22'])
        out['r31'] = F.relu(self.conv3_1(out['p2']))
        out['r32'] = F.relu(self.conv3_2(out['r31']))
        out['r33'] = F.relu(self.conv3_3(out['r32']))
        out['r34'] = F.relu(self.conv3_4(out['r33']))
        out['p3'] = self.pool3(out['r34'])
        out['r41'] = F.relu(self.conv4_1(out['p3']))
        out['r42'] = F.relu(self.conv4_2(out['r41']))
        out['r43'] = F.relu(self.conv4_3(out['r42']))
        out['r44'] = F.relu(self.conv4_4(out['r43']))
        out['p4'] = self.pool4(out['r44'])
        out['r51'] = F.relu(self.conv5_1(out['p4']))
        out['r52'] = F.relu(self.conv5_2(out['r51']))
        out['r53'] = F.relu(self.conv5_3(out['r52']))
        out['r54'] = F.relu(self.conv5_4(out['r53']))
        out['p5'] = self.pool5(out['r54'])
        return [out[key] for key in out_keys]

[VGG 구조 패턴]

- Block 1: 3→64→64 (얕은 특징: 선, 모서리)
- Block 2: 64→128→128 (중간 특징: 질감)
- Block 3: 128→256→256→256 (깊은 특징: 패턴)
- Block 4: 256→512→512→512 (더 복잡한 특징)
- Block 5: 512→512→512→512 (추상적 특징)

In [ ]:
vgg = VGG()

img1 = "vangogh_starry_night.jpg"
img2 = "Tuebingen_Neckarfront.jpg"

img1 = Image.open(img1)
img2 = Image.open(img2)
imgs = []
imgs.append(img1)
imgs.append(img2)

img_size = 512
prep = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),  # Tensor (c h w) [0 - 1] 
            transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), # RGB >> BGR
            transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961],
                                 std=[1,1,1]), # ImageNet  평균값 정규화
            transforms.Lambda(lambda x: x.mul_(255)),
            # pixel [0,255] 스케일 >> 픽셀 크기를 VGG 입력 스케일에 맞춤
        ])

# PIL -> tensor
# imgs 이미지 리스트 전체
imgs_torch = [prep(img) for img in imgs]

if torch.cuda.is_available():
    # imgs_torch = [Variable(img.unsqueeze(0).cuda()) for img in img_torch]
    imgs_torch = [img.unsqueeze(0).cuda() for img in imgs_torch]
else:
    imgs_torch = [img.unsqueeze(0) for img in imgs_torch]

style_image, content_image = imgs_torch

# 형태 유지하며 스타일만 변형되도록 학습(SGD) 시작
# 이미지 학습(이미지 개별픽셀 학습O) <> 이미지 특징학습X
# 스타일 전이학습에서 CNN 가중치 학습X
# content_image는 채점 기준표
# opt_img = Variable(content_image.data.clone(), requires_grad=True)
opt_img = content_image.clone().detach().requires_grad_(True)

'\nopt_img = content_image.clone().detach().requires_grad_(True)\n# content_image (데이터, 값) 복제해서 기존 그래프 분리(detach) >> 기울기 계산\n'

In [ ]:
import torch
import torch.nn as nn

if torch.cuda.is_available():
    vgg = vgg.cuda()
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# 스타일 손실을 계산할 VGG 레이어 이름 정의
# style_layers : VGG 내부의 특정 합성곱 레이어

style_layers = ['r11','r21','r31','r41', 'r51']

# 콘텐츠 손실을 계산할 VGG 레이어 이름 정의
# (일반적으로 중간정도 레이어 중 하나 사용)
content_layers = ['r42']

loss_layers = style_layers + content_layers

# 각 스타일에 대해 GramMSELoss 모듈을 사용하고, 콘텐츠에 대해 MSELoss 모듈을 사용하도록 리스트를 정의함.
# GramMSELoss 모듈 : 스타일 손실(loss) 계산
# [GramMSELoss, GramMSELoss, ..., nn.MSELoss] 형태가 됨.

loss_fns = [GramMSELoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)

if torch.cuda.is_available():
    # loss_fns 리스트의 요소들을 새로운 모듈 인스턴스로 만들고 .cuda()를 적용
    # 리스트 복사를 방지하고 정확하게 GPU로 이동
    loss_fns = [GramMSELoss().to(device) for _ in style_layers] + \
               [nn.MSELoss().to(device) for _ in content_layers]
    #  for _ in style_layers : style layers 개수만큼 GramMSELoss() 넣어요 >> 인스턴스
    #  >> 각각 새로 생성
else:
    # CPU 사용 시에도 동일한 로직으로 인스턴스화
    loss_fns = [GramMSELoss().to(device) for _ in style_layers] + \
               [nn.MSELoss().to(device) for _ in content_layers]

# 스타일 손실에 적용할 가중치(beta)를 정의
# 깊은 레이어일수록(복잡한 패턴을 추출하는 레이어) 낮은 가중치를 주는 경향이 있음
# 왜? 가중치가 감소되니깐
style_weights = [1e3/n**2 for n in [64,128,256,512,512]]
# [64,128,256,512,512] : channel(c) 수
# 1e3/n**2 : 보정 값. 깊은 레이어일 수로 feature 수가 많고 값도 커지므로 스타일 손실이 너무 커지는 거 방지

# 콘텐츠 손실에 부여할 가중치(alpha)를 정의
content_weights = [1e0]
# 1e0 : 1

weights = style_weights + content_weights

# 최적화 목표값 (style targets) 계산
# style_image >> VGG 통과 >> 각 스타일 레이어 Gram Matrix 계산
# >> 변화도 추적에서 제외 (detach)
style_targets = [GramMatrix()(A).detach() for A in vgg(style_image, style_layers)]
# vgg(style_image, style_layers) 지정한 레이어들의 특징맵(feature map )리스트로 반환
# [A_r11, A_r21.....] 여기서 A.shape (b,c,h,w)
# GramMatrix(A) >> 형태 변환 (b, c, c) >> 스타일 표현
# .detach() 계산 그래프 분리(역전파시 gradient 계산되지 않도록)
# >> 왜? style_targets 고정된 값(ground truth)
# style_targets? 각 스타일 레이어에 대한 스타일 이미지 Gram Matrix 목록

# 최적화 목표값(content targets)을 계산함.
# 콘텐츠 이미지(content_image)를 VGG에 통과시켜 콘텐츠 레이어의 특징 맵을 추출하고 변화도 추적에서 제외(detach)했음.
# content_image 또한 이미 .cuda() 또는 .to(device)로 GPU에 로드되어 있다고 가정합니다.
content_targets = [A.detach() for A in vgg(content_image, content_layers)]

# 최종적으로 사용할 모든 목표값 리스트를 정의함.
targets = style_targets + content_targets


In [9]:
targets

[tensor([[[317.9097,  14.2340,   9.5391,  ...,  30.3477, 125.0743,  30.5910],
          [ 14.2340, 539.9493,  55.6673,  ...,  72.0618,  10.4826, 608.8537],
          [  9.5391,  55.6673,  41.4998,  ...,  77.6339,   4.3284,  68.2613],
          ...,
          [ 30.3477,  72.0618,  77.6339,  ..., 575.5036,  22.8726,  35.3250],
          [125.0743,  10.4826,   4.3284,  ...,  22.8726,  65.3410,   9.6822],
          [ 30.5910, 608.8537,  68.2613,  ...,  35.3250,   9.6822, 791.8442]]],
        device='cuda:0'),
 tensor([[[3.2912e+00, 1.6034e-02, 4.9145e+00,  ..., 7.2559e-03,
           7.6530e-04, 8.2010e-01],
          [1.6034e-02, 2.4941e-01, 2.2831e-01,  ..., 1.6959e-02,
           4.3480e-05, 4.6047e-01],
          [4.9145e+00, 2.2831e-01, 1.4602e+01,  ..., 7.5723e-01,
           6.7991e-03, 4.6019e+00],
          ...,
          [7.2559e-03, 1.6959e-02, 7.5723e-01,  ..., 5.8756e-01,
           4.1775e-04, 9.8683e-01],
          [7.6530e-04, 4.3480e-05, 6.7991e-03,  ..., 4.1775e-04,
     

In [10]:
input_image = content_image.clone().requires_grad_(True)
# input_image 는 VGG에 입력될 초기 이미지, 콘텐츠 이미지와 동일해야 함
input_image

tensor([[[[  15.0610,   19.0610,   32.0610,  ...,   68.0610,   71.0610,
             70.0610],
          [  10.0610,   16.0610,   16.0610,  ...,   69.0610,   70.0610,
             69.0610],
          [   6.0610,   15.0610,   23.0610,  ...,   70.0610,   71.0610,
             70.0610],
          ...,
          [ -81.9390,  -83.9390,  -84.9390,  ...,    5.0610,   -2.9390,
             -0.9390],
          [ -82.9390,  -85.9390,  -85.9390,  ...,   -8.9390,   -2.9390,
             -5.9390],
          [ -83.9390,  -88.9390,  -87.9390,  ...,   -7.9390,   -6.9390,
            -15.9390]],

         [[ -33.7790,  -32.7790,  -21.7790,  ...,   -2.7790,    0.2210,
             -0.7790],
          [ -33.7790,  -30.7790,  -34.7790,  ...,   -1.7790,   -0.7790,
             -1.7790],
          [ -31.7790,  -26.7790,  -21.7790,  ...,   -0.7790,    0.2210,
             -0.7790],
          ...,
          [ -91.7790,  -93.7790,  -94.7790,  ...,   19.2210,   11.2210,
             14.2210],
          [ -92.77

In [ ]:
# L-BFGS 옵티마이저: 최적화 단계에서 closure 함수 요구
optimizer = torch.optim.LBFGS([input_image], max_iter=1)
# LBFGS([초기 이미지]) : CNN 가중치가 아니라 이미지 픽셀 학습
# LBFGS (Quasi Newton optimizer) : 고차원 연속계산 적합(고품질 결과물)

# optimizer = torch.optim.Adam([input_image], lr=0.01)
# Adam 사용 가능 : 일반적으로 안정되면서 빠른 테스트 결과

# 최적화 함수 세기 위한 변수
n_iter = 0

In [ ]:
def closure():
    global n_iter
    optimizer.zero_grad()

    # 생성된 이미지(input_image)를 VGG에 통과시켜 특징 맵을 추출합니다.
    # vgg는 이전에 정의되어 GPU로 이동되었다고 가정합니다.
    # input_image >> feature map 목록 [A_r11, A_r21, ...]
    out = vgg(input_image, loss_layers)

    # 총 손실을 계산합니다.
    layer_losses = []
    total_loss = 0

    # 각 레이어별 손실(콘텐츠 손실, 스타일 손실) 계산
    # 가중치 곱
    # 전체 손실에 더함
    for i, weight in enumerate(weights):
        target = targets[i]     # 스타일 이미지 / 콘텐츠 이미지 타겟
        feature = out[i]        # 현재 생성 이미지의 feature map
        loss_fn = loss_fns[i]   # 해당 레이어(GramMSELoss, MSELoss)
        # GramMatrix 계산이 필요한 스타일 레이어 처리 (GramMSELoss가 GramMatrix를 내부에서 처리한다고 가정)
        # GramMatrix()가 별도 모듈이면, GramMSELoss 내부에 GramMatrix가 포함되어 있어야 합니다.

        loss = weight * loss_fn(feature, target)
        layer_losses.append(loss.item())
        total_loss += loss

    total_loss.backward() # input 이미지에 대해 계산

    if n_iter % 50 == 0:
        print(f"Iteration {n_iter}: Total Loss = {total_loss.item():.4f}")
        # print(f"Layer Losses: {layer_losses}") # 디버깅용

    n_iter += 1
    return total_loss

# 최적화 실행 (반복 횟수 지정)
num_iterations = 500 # 원하는 반복 횟수를 설정합니다.
for i in range(num_iterations):
    # L-BFGS는 step() 호출 시마다 클로저를 여러 번 호출할 수 있습니다.
    # closure 반환값 기반 처리
    optimizer.step(closure)

    # closure() >> total_loss 반환

    # 참고: Adam을 사용한다면, 루프는 다음과 같습니다.
    # loss = closure()
    # optimizer.step()

# 최종 결과 이미지 후처리 (옵션) 이미지 픽셀값 정규화
# 생성된 이미지를 [0, 1] 범위로 클리핑하여 픽셀 값을 보정합니다.
input_image.data.clamp_(0, 1)

# 최적화된 input_image.data가 최종 스타일 트랜스퍼 결과입니다.

Iteration 0: Total Loss = 8689.9297
Iteration 50: Total Loss = 0.0946
Iteration 100: Total Loss = 0.0051
Iteration 150: Total Loss = 0.0020
Iteration 200: Total Loss = 0.0018
Iteration 250: Total Loss = 0.0018
Iteration 300: Total Loss = 0.0018
Iteration 350: Total Loss = 0.0018
Iteration 400: Total Loss = 0.0018
Iteration 450: Total Loss = 0.0018


tensor([[[[1., 0., 0.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          ...,
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.]]]], device='cuda:0')